# Teoria da Dualidade

## Primal

Max: $z = 4x_1 + x_2 + 5x_3 + 3x_4$

S.a: 

\begin{align}
x_1 - x_2 - x_3 + 3x_4 \le 1 \\
5x_1 + x_2 + 3x_3 + 8x_4 \le 55 \\
-x_1 + 2x_2 + 3x_3 - 5x_4 \le 3 \\
x_1, x_2, x_3, x_4 \ge 0
\end{align}

In [1]:
#include<iostream>

In [2]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

In [3]:
  // 1. Escolha o solver.
  std::unique_ptr<MPSolver> solver(MPSolver::CreateSolver("Primal", "GLOP"));

In [4]:
  // 2. Crie as variáveis: 
  auto const x1 = solver->MakeNumVar(0.0, solver->infinity(), "x1");
  auto const x2 = solver->MakeNumVar(0.0, solver->infinity(), "x2");
  auto const x3 = solver->MakeNumVar(0.0, solver->infinity(), "x3");
  auto const x4 = solver->MakeNumVar(0.0, solver->infinity(), "x4");
  std::cout << "Número de variáveis = " << solver->NumVariables() << std::endl;

Número de variáveis = 4


In [5]:
  // 3. Defina as restrições: 
  auto const ct1 = solver->MakeRowConstraint(-solver->infinity(), 1.0, "ct1");
  ct1->SetCoefficient(x1, 1.0);
  ct1->SetCoefficient(x2, -1.0);
  ct1->SetCoefficient(x3, -1.0);
  ct1->SetCoefficient(x4, 3.0);

  auto const ct2 = solver->MakeRowConstraint(-solver->infinity(), 55.0, "ct2");
  ct2->SetCoefficient(x1, 5.0);
  ct2->SetCoefficient(x2, 1.0);
  ct2->SetCoefficient(x3, 3.0);
  ct2->SetCoefficient(x4, 8.0);

  auto const ct3 = solver->MakeRowConstraint(-solver->infinity(), 3.0, "ct3");
  ct3->SetCoefficient(x1, -1.0);
  ct3->SetCoefficient(x2, 2.0);
  ct3->SetCoefficient(x3, 3.0);
  ct3->SetCoefficient(x4, -5.0);

  std::cout << "Número de restrições = " << solver->NumConstraints() << std::endl;

Número de restrições = 3


In [6]:
  // 4. Defina a função objetivo: 
  auto const objective = solver->MutableObjective();
  objective->SetCoefficient(x1, 4.0);
  objective->SetCoefficient(x2, 1.0);
  objective->SetCoefficient(x3, 5.0);
  objective->SetCoefficient(x4, 3.0);
  objective->SetMaximization();

In [7]:
  // 5. Execute o solver e exiba os resultados.
  auto status = solver->Solve();

In [8]:
  if (status == MPSolver::OPTIMAL)
  {
      std::cout << "Solução:" << std::endl;
      std::cout << "Valor da função objetivo = " << objective->Value() << std::endl;
      std::cout << "x1 = " << x1->solution_value() << std::endl;
      std::cout << "x2 = " << x2->solution_value() << std::endl;
      std::cout << "x3 = " << x3->solution_value() << std::endl;
      std::cout << "x4 = " << x4->solution_value() << std::endl;
  }

Solução:
Valor da função objetivo = 29
x1 = 0
x2 = 14
x3 = 0
x4 = 5


## Dual

Min: $w = y_1 + 55y_2 + 3y_3$

S.a: 

\begin{align}
 y_1 + 5y_2 -  y_3 \ge 4 \\
-y_1 +  y_2 + 2y_3 \ge 1 \\
-y_1 + 3y_2 + 3y_3 \ge 5 \\
3y_1 + 8y_2 - 5y_3 \ge 3 \\
y_1, y_2, y_3 \ge 0
\end{align}

In [9]:
  // 1. Escolha o solver.
  std::unique_ptr<MPSolver> solver(MPSolver::CreateSolver("Dual", "GLOP"));

In [10]:
  // 2. Crie as variáveis: 
  const MPVariable * y[3];
  for(int i=0; i < 3; i++)
    y[i] = solver->MakeNumVar(0.0, solver->infinity(), "");
  std::cout << "Número de variáveis = " << solver->NumVariables() << std::endl;

Número de variáveis = 3


In [11]:
  // 3. Defina as restrições: 
  auto const e1 = +1.0 * LinearExpr(y[0]) + 5.0 * LinearExpr(y[1]) - 1.0 * LinearExpr(y[2]);
  auto const e2 = -1.0 * LinearExpr(y[0]) + 1.0 * LinearExpr(y[1]) + 2.0 * LinearExpr(y[2]);
  auto const e3 = -1.0 * LinearExpr(y[0]) + 3.0 * LinearExpr(y[1]) + 3.0 * LinearExpr(y[2]);
  auto const e4 = +3.0 * LinearExpr(y[0]) + 8.0 * LinearExpr(y[1]) - 5.0 * LinearExpr(y[2]);

  auto const ct1 = solver->MakeRowConstraint(e1 >= 4.0);
  auto const ct2 = solver->MakeRowConstraint(e2 >= 1.0);
  auto const ct3 = solver->MakeRowConstraint(e3 >= 5.0);
  auto const ct4 = solver->MakeRowConstraint(e4 >= 3.0);

  std::cout << "Número de restrições = " << solver->NumConstraints() << std::endl;

Número de restrições = 4


In [12]:
  // 4. Defina a função objetivo: 
  auto const objective = solver->MutableObjective();
  objective->MinimizeLinearExpr(1.0 * LinearExpr(y[0]) + 55.0 * LinearExpr(y[1]) + 3.0 * LinearExpr(y[2]));

In [13]:
  // 5. Execute o solver e exiba os resultados.
  auto status = solver->Solve();
  if (status == MPSolver::OPTIMAL)
  {
      std::cout << "Solução:" << std::endl;
      std::cout << "Valor da função objetivo = " << objective->Value() << std::endl;
      std::cout << "y1 = " << y[0]->solution_value() << std::endl;
      std::cout << "y2 = " << y[1]->solution_value() << std::endl;
      std::cout << "y3 = " << y[2]->solution_value() << std::endl;
  }

Solução:
Valor da função objetivo = 29
y1 = 11
y2 = 0
y3 = 6


In [14]:
ct1->dual_value()//x1

0.0000000

In [15]:
ct2->dual_value()//x2

14.000000

In [16]:
ct3->dual_value()//x3

0.0000000

In [17]:
ct4->dual_value()//x4

5.0000000